In [ ]:
import torch
import torch.nn as nn
import fastai.layers as L
import torch.nn.functional as F
from collections import OrderedDict

torch.backends.cudnn.deterministic = True

In [ ]:
def Embedding(weight_matrix):

  num_embeddings, dim_embeddings = weight_matrix.shape
  weights_matrix = torch.from_numpy(weights_matrix)

  Emb_layer = nn.Embedding(num_embeddings, dim_embeddings, padding_idx=1).to('cuda')
  Emb_layer.load_state_dict({'Weight' : weight_matrix})
  Emb_layer.weight.requires_grad = False

  return Emb_layer

In [ ]:
class downsample_merger(nn.Module):
  def __init__(self, input_channel, dense = True):
    super().__init()
    self.downsample = L.conv1d(input_channel, input_channel * 2, stride = 2)

  def forward(self, x): return x + self.downsample(x.orig)

In [ ]:
def transition(input_channel):
  trans_layer = L.SequentialEX(
      L.conv1d(input_channel, input_channel * 2, ks = 3, stride = 2, padding = 1),
      downsample_merger(input_channel)
  )
  return trans_layer

In [ ]:
def residual(input_channel, leaky):
  kwargs = {'is_1d' : True, 'self_attention' : True, 'leaky' : leaky}
  return L.res_block(input_channel, bottle = True, **kwargs)

In [ ]:
class DeepResNet(nn.Module):
  def __init__(self, weights_matrix, layers = (4, 4, 7, 7),
               input_channel = 16, dim_embed = 50, leaky = 0.02, dropout = 0.25):
    super(DeepResNet, self).__init__()

    self.embedding = Embedding(weight_matrix)
    self.init_conv = L.conv1d(1, input_channel, ks = (3, dim_embed),
                              stride = 1, padding = (1, 0), bias = False)
    self.dropout = dropout
    
    self.features = nn.Sequential(OrderedDict([
                                                ('initial_conv', self.init_conv),
                                                ('init_norm', nn.BatchNorm1d(input_channel)),
                                                ('init_LeakyReLU', nn.LeakyReLU(inplace = True))
    ]))

    num_features = inplanes

    for count, layer in enmerate(layers):
      self.features.add_module('residual #', count + 1, self.initial_block(num_features, layer))

      if count < len(layers):
        self.features.add_module('transition #', count + 1, transition(num_features))
        num_features = num_features * 2

    self.leakyrelu = L.relu(inplace = True, leaky = leaky)
    self.maxpool = nn.AdaptiveMaxPool1d(1)
    self.FC1 = nn.Linear(num_features, num_features * 2)
    self.classifier = nn.Linear(num_features * 2, 2)

    for module in self.modules():
      if isinstance(module, nn.BatchNorm1d):
        nn.init.xavier_uniform_(module.weight, 1)
        nn.init.xavier_uniform_(module.bias, 0)
      elif isinstance(module, nn.Linear):
        nn.init.xavier_uniform_(module.bias, 0)
  
  def initial_block(self, output_channel, layer_counts, leaky):
    layers = []

    for counts in range(0, layer_counts):
      
      layers.append(residual(output_channel, leaky))
      if counts < layer_count: layers.appned(nn.Dropout(p = self.dropout, inplace = True))

    return nn.Sequential(*layers)

  def forward(self, x):

    x = self.embedding(x).unsqueeze(1)

    for count, layers in enumarate(self.features):
      if count is 1: x = x.squeeze(3)
      x = layer(x)

    x = self.maxpool(x).view(x.size(0), -1)
    x = self.FC1(x)
    x = self.leakyrelu(x)
    x = self.classifier(x)

    return x